In [ ]:
import re
import requests
from bs4 import BeautifulSoup
bacteria_regex = r'[A-Z]\. [a-z]+'
bacteria_mic_regex = r'MIC ' + bacteria_regex
bacteria_equals_regex = '(?P<bacterium>' + bacteria_regex + r')\s?=\s?' + '(?P<value>\d+\.?\d*)'
DATABASE_NAME = 'YADAMP'

In [ ]:
i = 2222
url_base = 'http://www.yadamp.unisa.it/showItem.aspx?yadampid='
url = url_base + str(i)
content = requests.get(url).content
soup = BeautifulSoup(content, 'html.parser')

In [ ]:
amps = {}
NUM_AMPS = 2525

In [ ]:
def get_references(table):
    for table_row in table.findAll('tr'):
        rows = table_row.findAll('td')
        for i, row in enumerate(rows):
            if row.string and 'TITLE PAPER' in row.string:
                reference_cell = rows[i+1]
    reference_string = reference_cell.text.strip()
    reference_link = reference_cell.find('a').get('href') if reference_cell.find('a') else ''
    return [reference_string + ' | ' + reference_link]

In [ ]:
for i in range(max(len(amps), 1), NUM_AMPS):
    url_base = 'http://www.yadamp.unisa.it/showItem.aspx?yadampid='
    url = url_base + str(i)
    content = requests.get(url).content
    soup = BeautifulSoup(content, 'html.parser')
    table = soup.find('table')

    bacteria_dict = {}
    UNIT = 'uM'
    sequence = None
    for table_row in table.findAll('tr'):
        rows = table_row.findAll('td')
        for j, row in enumerate(rows):
            if not row.string:
                continue
            if re.search(bacteria_mic_regex, row.string):
                bacterium = re.search(bacteria_regex, row.string).group(0)
                value = rows[j+1].string.strip()
                if value:
                    bacteria_dict[(bacterium, None)] = {
                        'value': value,
                        'unit': UNIT
                    }
            elif re.search(bacteria_equals_regex, row.string):
                for match in re.finditer(bacteria_equals_regex, row.string):
                    groupdict = match.groupdict()
                    bacterium = groupdict['bacterium']
                    value = groupdict['value']
                    bacteria_dict[(bacterium, None)] = {
                        'value': value,
                        'unit': UNIT
                    }
            elif row.string.strip() == 'SEQUENCE':
                sequence_row = rows[j+1]
                sequence = list(sequence_row.findAll('b')[0].children)[0]
    if not sequence:
        pass # print('Error sequence not found', i, sequence)
    else:
        # print("Adding sequence:", i, sequence)
        amps[sequence] = {}
        amps[sequence]['bacteria'] = bacteria_dict
        amps[sequence]['references'] = get_references(table)
        amps[sequence]['url_sources'] = [url]
    if i % 50 == 0:
        print(i)

In [ ]:
with open(DATABASE_NAME + ".data", 'w') as f:
    f.write(str(amps))